# 6-Cluster Geofencing

In this notebook we use the results from the previous one to display the geographic clusters on a map, using the H3 package.

**Requirements:**

- Please run the `05-clustering-hdbscan.ipynb` notebook first and its dependencies.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html). Enable using `jupyter nbextension enable --py widgetsnbextension --sys-prefix` for Jupyter Notebook and `jupyter labextension install @jupyter-widgets/jupyterlab-manager` for Jupyter Lab.

In [ ]:
import numpy as np
import folium
import ipywidgets as widgets
import networkx as nx

from folium.vector_layers import PolyLine
from sqlapi import VedDb
from h3 import h3
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
from ipywidgets import interact
from tqdm.auto import tqdm

Create an object of the `VedDB` type to interface with the database.

In [ ]:
db = VedDb()

To illustrate the process of retrieval of an endpoint's hexagons, let's run a simple query to return all the H3 codes for cluster number 23.

In [ ]:
def get_cluster_hexes(cluster_id):
    sql = "select h3 from cluster_point where cluster_id = ?"
    hexes = list({h[0] for h in db.query(sql, [cluster_id])})
    return hexes

In [ ]:
hexes = get_cluster_hexes(23)
hexes

The code above uses a Python `set` comprehension in order to retrieve the unique H3 codes. As you probably guessed, it is very likely to find overlapping H3 hexagons in the same cluster, and using a `set` eliminates repetitions.

To convert an H3 code into a map object, we must first expand it into a set of six geo locations using the `h3_to_geo_boundary` function.

In [ ]:
h = hexes[0]

The `get_hexagon` converts an H3 index into the corresponding hexagon vertices' geographic coordinates. Note how we copy the first coordinate to the back of the list in order to _close_ the polygon.

In [ ]:
def get_hexagon(h):
    geo_lst = list(h3.h3_to_geo_boundary(h))
    geo_lst.append(geo_lst[0])
    return np.array(geo_lst)

Let's see how the function works.

In [ ]:
hexagon = get_hexagon(h)
hexagon

## Display the Hexagon on the Map

To display the hexagon on a map, we use code that is similar to the one we used in notebook number 4 to display a trip.

Let's start by declaring the map object.

In [ ]:
html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron")

Determine the shape's bounding box and fit the map view to it.

In [ ]:
min_lat, max_lat = hexagon[:, 0].min(), hexagon[:, 0].max()
min_lon, max_lon = hexagon[:, 1].min(), hexagon[:, 1].max()
html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

Now, we create a `PolyLine` object and place it on the map.

In [ ]:
color = '#3388ff'
opacity = 0.7
polyline = PolyLine(hexagon, color=color, opacity=opacity, fill=color)
p = polyline.add_to(map)

In [ ]:
html_map

## Display the Cluster on the Map

We can now extend the code to the whole cluster and display all the hexagons side-by side.

In [ ]:
html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron")

bb_list = []  # List for the bounding-box calculation

for h in hexes:
    hexagon = get_hexagon(h)
    bb_list.extend(hexagon.tolist())
    polyline = PolyLine(hexagon, color=color, opacity=opacity, fill=color)
    p = polyline.add_to(html_map)
    
locations = np.array(bb_list)
min_lat, max_lat = locations[:, 0].min(), locations[:, 0].max()
min_lon, max_lon = locations[:, 1].min(), locations[:, 1].max()
html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
html_map

But we can still do better: how about only displaying the cluster outline? We can do so by merging all the hexagons together using Shapely's `Polygon` object and the `cascaded_union` function. The idea is to model each hexagon as a `Polygon` and then merge them into a single map polygon.

In [ ]:
def create_map_polygon(xy, tooltip='',
                       color='#3388ff',
                       opacity=0.7,
                       fill_color='#3388ff',
                       fill_opacity=0.4, 
                       weight=3):
    points = [[x[0], x[1]] for x in xy]
    polygon = folium.vector_layers.Polygon(locations=points,
                                           tooltip=tooltip,
                                           fill=True,
                                           color=color,
                                           fill_color=fill_color,
                                           fill_opacity=fill_opacity,
                                           weight=weight,
                                           opacity=opacity)
    return polygon

Start by creating the map with the whitewashed tiles.

In [ ]:
html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron")

Now, generate the H3 hexagons and convert them into Shapely `Polygon`s.

In [ ]:
bb_list = []  # List for the bounding-box calculation
polygons = []
for h in hexes:
    points = get_hexagon(h)
    xy = [[x[1], x[0]] for x in points]
    xy.append([points[0][1], points[0][0]])
    polygons.append(Polygon(xy))
    bb_list.extend(points)

With a single call to `cascaded_union`, we merge all the `Polygon` objects into a single one that we can conveniently plot on the map. Note that by converting all hexagons into a single polygon, we are not only making the display cleaner, but we are also reducing the amount of redundant points used to define the shape. If you decide to store the shape on a database, this will prove extremely useful.

In [ ]:
merged = cascaded_union(polygons)

The polygon merge operation may have created a complex polygonal shape with "holes". To gracefully handle this situation, we must check if the generated object is a simple `Polygon` or a `MultiPolygon`. The former case is simple to handle, but for the latter we simply use the largest one.

In [ ]:
if merged.geom_type == "MultiPolygon":
    max_len = 0
    largest = None
    for geom in merged.geoms:
        xy = geom.exterior.coords.xy
        lxy = list(zip(xy[1], xy[0]))

#         if len(lxy) > max_len:
#             max_len = len(lxy)
#             largest = lxy

        create_map_polygon(lxy).add_to(html_map)
elif merged.geom_type == "Polygon":
    xy = merged.exterior.coords.xy
    lxy = list(zip(xy[1], xy[0]))

    create_map_polygon(lxy).add_to(html_map)

We can now fit the shape boundaries to the map and display it.

In [ ]:
locations = np.array(bb_list)
min_lat, max_lat = locations[:, 0].min(), locations[:, 0].max()
min_lon, max_lon = locations[:, 1].min(), locations[:, 1].max()
html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
html_map

If you hover the mouse cursor over the shape you will see an empty tooltip. This feature would be interesting to use should we be able to automatically name these clusters, using real street name information. This is the challenge for the next notebook.

Meanwhile, let's check the quality of the generated polygons.

## Interactive Cluster Exploration


In [ ]:
def show_geofence_map(cluster_id):
    html_map = folium.Map(prefer_canvas=True, tiles="cartodbpositron")
    bb_list = []  # List for the bounding-box calculation
    polygons = []
    hexes = get_cluster_hexes(cluster_id)
    for h in hexes:
        points = get_hexagon(h)
        xy = [[x[1], x[0]] for x in points]
        xy.append([points[0][1], points[0][0]])
        polygons.append(Polygon(xy))
        bb_list.extend(points)
        
    merged = cascaded_union(polygons)
    
    if merged.geom_type == "MultiPolygon":
        max_len = 0
        largest = None
        for geom in merged.geoms:
            xy = geom.exterior.coords.xy
            lxy = list(zip(xy[1], xy[0]))
            create_map_polygon(lxy).add_to(html_map)
    elif merged.geom_type == "Polygon":
        xy = merged.exterior.coords.xy
        lxy = list(zip(xy[1], xy[0]))

        create_map_polygon(lxy).add_to(html_map)
        
    locations = np.array(bb_list)
    min_lat, max_lat = locations[:, 0].min(), locations[:, 0].max()
    min_lon, max_lon = locations[:, 1].min(), locations[:, 1].max()
    html_map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    return html_map

In [ ]:
sql = "select count(distinct cluster_id) from cluster_point"
cluster_count = db.query_scalar(sql)

In [ ]:
cluster_count

In [ ]:
ii = interact(show_geofence_map, cluster_id=widgets.IntSlider(min=0, max=cluster_count-1, step=1, value=0))

It seems that not all clusters are represented by a single shape, and it is not a rare thing. What went wrong here?

By selecting a fixed size for the H3 hexagons, we are actually enabling that these gaps show up when a particular set of points strays farther than the hexagon's size. How can we solve this issue? We can either increase the size of the hexagons, or we can be a bit smarter and build _bridges_ between the _islands_. But for that, we need the help from graph theory.

## Clusters as Networks

The idea is to analyze a specific cluster and determine if all the hexagons are neighbors. A network is useful here to determine if there are isolated hexagons. First we create a network using `networkx` and feed it with all the hexagon codes as nodes. Next, for each pair of neighboring hexagons, we declare an edge on the network. Once this is done, we can compute the _connected components_ and use that as a means to connect them back.

The function `graph_from_hexes` accepts a list od H3 indexes and build a graph of their neighboring status.

In [ ]:
def graph_from_hexes(hexes):
    g = nx.Graph()
    for i in range(len(hexes)):
        g.add_node(hexes[i])
        for j in range(i+1, len(hexes)):
            if h3.h3_indexes_are_neighbors(hexes[i], hexes[j]):
                g.add_edge(hexes[i], hexes[j])
    return g

Now, let's build and display the graph for cluster `0`.

In [ ]:
hexes = get_cluster_hexes(0)
g = graph_from_hexes(hexes)
nx.draw(g)

We can now query how many connected components there are and which ones are they.

In [ ]:
nx.number_connected_components(g)

In [ ]:
components = list(nx.connected_components(g))
components

To make our lives easier, `networkx` reports the connected components as a list of sets. But now we need to build the _bridge_ between the components, and we will use H3 for that. The idea is to use the `h3_line` function and enumerate all possible ways we have to connect the hexagons between the two components. We will keep the smallest line that should, in the best scenarios, have only 3 hexagons, and which is actually a very good early stopping criterion. Note that in some circumstances we might need more hexagons to bridge the gap.

Let's see how the function works:

In [ ]:
h3.h3_line('8c274994c85a3ff', '8c274994cbb45ff')

Here's the draft of the algorithm below.

In [ ]:
bridges = []
for h0 in components[0]:
    for h1 in components[1]:
        bridge = h3.h3_line(h0, h1)
        bridges.append(bridge)

We can now inspect the list of possible bridges and there three candidates. Take your pick!

In [ ]:
bridges

We can ask Python to do the work automatically for us:

In [ ]:
min(bridges, key=lambda v: len(v))

As you can see, both the first and last elements of the list already belong to the connected components. To patch up the cluster geofence, we just need the middle H3 index, and add it to the cluster.

Below, we automate the process for all clusters by adding the missing hexagons to the clusters, not forgetting to store them in the database.

In [ ]:
def get_bridge(components):
    bridges = []
    for h0 in components[0]:
        for h1 in components[1]:
            bridge = h3.h3_line(h0, h1)
            
            if len(bridge) == 3:
                # This is a good solution for sure
                return bridge
            bridges.append(bridge)
    return min(bridges, key=lambda v: len(v))

In [ ]:
for cluster_id in tqdm(range(cluster_count)):
    hexes = get_cluster_hexes(cluster_id)
    g = graph_from_hexes(hexes)
    
    while nx.number_connected_components(g) > 1:
        components = list(nx.connected_components(g))
        bridge = get_bridge(components)
        
        pts = []
        for h in bridge[1:-1]:
            lat_lon = h3.h3_to_geo(h)
            hexes.append(h)
            pts.append((cluster_id, lat_lon[0], lat_lon[1], h))
            
        db.insert_cluster_points(pts)
        
        # Rebuild the graph
        g = graph_from_hexes(hexes)

You can now go back to the interactive map and explore the patched geofences.

In the next notebook we will try to give sensible names to these babies...